In [1]:
from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph,START,END
from typing import TypedDict
from dotenv import  load_dotenv
load_dotenv()

True

In [2]:
! ollama list

NAME                                   ID              SIZE      MODIFIED     
dimavz/whisper-tiny:latest             9aafc61ff108    44 MB     34 hours ago    
lirockyzhang/orpheus-3b-0.1-ft:q4_k    0d255ddcbb3a    2.4 GB    37 hours ago    
gemma3:4b                              a2af6cc3eb7f    3.3 GB    2 days ago      
deepseek-r1:8b                         6995872bfe4c    5.2 GB    2 days ago      
phi4-mini-reasoning:3.8b               3ca8c2865ce9    3.2 GB    9 days ago      
nomic-embed-text-v2-moe:latest         ff9c2f10ef5e    957 MB    10 days ago     
qwen3:8b                               500a1f067a9f    5.2 GB    10 days ago     


In [3]:
groq=ChatGroq(model='llama-3.1-8b-instant')
ollama=ChatOllama(model='deepseek-r1:8b')


In [4]:
class LLMState(TypedDict):
    question:str
    answer:str

    

In [64]:
def LLM_QA(state:LLMState)->LLMState:
    question=state['question']
    prompt =f'Answer the following {question} and provide the answer in very precise and crisp format and dont try to justify the answer very long'
    answer=ollama.invoke(prompt).content
    state['answer']=answer
    return state

In [61]:
graph = StateGraph(LLMState)

#graph.add_node(name,node_fn)
graph.add_node('llm_qa',LLM_QA)
graph.add_edge(START,'llm_qa')
graph.add_edge('llm_qa',END)
flow=graph.compile()

In [101]:
initial_state={'question':"How far is the moon from china"}

answer=flow.invoke(initial_state)
print(answer)

{'question': 'How far is the moon from china', 'answer': '**384,400 kilometers**'}


# Prompt Chaining

In [66]:
class BlogState(TypedDict):
    title:str
    outline:str
    content:str

In [97]:
def create_outline(state: BlogState) -> BlogState:

    # fetch title
    title = state['title']

    # call llm gen outline
    prompt = f'Generate a detailed outline for a blog on the topic - {title}'
    outline = groq.invoke(prompt).content

    # update state
    state['outline'] = outline

    return state

In [98]:
def create_blog(state: BlogState) -> BlogState:

    title = state['title']
    outline = state['outline']

    prompt = f'Write a detailed blog on the title - {title} using the follwing outline \n {outline}'

    content = groq.invoke(prompt).content

    state['content'] = content

    return state

In [102]:
graph = StateGraph(BlogState)

# nodes
graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)

# edges
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', END)

workflow = graph.compile()

In [104]:
initial_state={"title":"Rise Of Political Hatred between communites in India"}
workflow.invoke(initial_state)

{'title': 'Rise Of Political Hatred between communites in India',
 'outline': '**Title:** "The Rise of Political Hatred between Communities in India: Understanding the Complexities and Consequences"\n\n**I. Introduction**\n\n* Brief overview of the rise of political hatred between communities in India\n* Importance of the topic and its relevance to Indian society\n* Thesis statement: The rise of political hatred between communities in India is a complex issue with deep historical, social, and economic roots that requires a nuanced understanding to address effectively.\n\n**II. Historical Context**\n\n* Overview of the partition of India and its impact on communal relations\n* The role of colonialism in shaping communal identities and conflicts\n* Key events and policies that have contributed to the rise of political hatred, such as:\n\t+ The Emergency of 1975\n\t+ The Ayodhya controversy\n\t+ The Gujarat riots of 2002\n\t+ The Kashmir conflict\n\n**III. Social and Economic Factors**\n\

# Parallel Workflow